# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_csv = '../output_data/cities.csv'
cities_df = pd.read_csv(cities_csv)
cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Leningradskiy,69.38,178.42,18.23,96,100,12.12,RU,1604362146
1,1,San Jose,37.34,-121.89,81.00,27,90,6.93,US,1604362072
2,2,Esperance,-33.87,121.90,59.00,100,75,10.29,AU,1604362147
3,3,Chokurdakh,70.63,147.92,-13.22,88,74,9.71,RU,1604362147
4,4,Santa Cruz Cabrália,-16.28,-39.02,68.00,88,90,6.93,BR,1604362147
...,...,...,...,...,...,...,...,...,...,...
560,560,Pulaski,34.77,-92.30,55.40,37,1,5.82,US,1604362410
561,561,Manakara,-22.13,48.02,67.05,82,20,3.40,MG,1604362410
562,562,Richards Bay,-28.78,32.04,67.44,93,29,7.63,ZA,1604362411
563,563,Zambezi,-13.54,23.10,71.10,45,51,2.98,ZM,1604362411


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 300, point_radius =5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Set ideal weather conditions for a vacation:

Temp: Greater than 75 but less than 90
Wind Speed: Less than 15 mph
Cloudiness: Less than 10%
Humidity: Less than 60%

In [4]:
vacation_df = cities_df.loc[(cities_df['Max Temp']>75)&(cities_df['Max Temp']<90)
                            &(cities_df['Wind Speed']<15)
                            &(cities_df['Cloudiness']<10)
                            &(cities_df['Humidity']<60)].dropna()

vacation_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,Hobart,-42.88,147.33,77.00,43,0,10.29,AU,1604361923
87,87,Verāval,20.90,70.37,78.69,41,0,13.76,IN,1604362184
160,160,Formosa,-26.18,-58.18,77.00,44,0,5.82,AR,1604362037
187,187,Santa Rosalía,27.32,-112.28,82.60,35,0,2.62,MX,1604362233
224,224,Alice Springs,-23.70,133.88,89.60,11,0,11.41,AU,1604362050
335,335,Broken Hill,-31.95,141.43,82.40,20,0,14.99,AU,1604362301
342,342,Laguna,38.42,-121.42,82.99,18,1,3.11,US,1604362305
389,389,Diu,20.71,70.98,76.86,45,0,11.25,IN,1604362328
444,444,Alice,27.75,-98.07,75.20,36,1,4.70,US,1604362354


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df=vacation_df
hotel_df['Hotel Name']=""
hotel_df = hotel_df.rename(columns={'Unnamed: 0': 'City ID'})
hotel_df

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
20,20,Hobart,-42.88,147.33,77.00,43,0,10.29,AU,1604361923,
87,87,Verāval,20.90,70.37,78.69,41,0,13.76,IN,1604362184,
160,160,Formosa,-26.18,-58.18,77.00,44,0,5.82,AR,1604362037,
187,187,Santa Rosalía,27.32,-112.28,82.60,35,0,2.62,MX,1604362233,
224,224,Alice Springs,-23.70,133.88,89.60,11,0,11.41,AU,1604362050,
335,335,Broken Hill,-31.95,141.43,82.40,20,0,14.99,AU,1604362301,
342,342,Laguna,38.42,-121.42,82.99,18,1,3.11,US,1604362305,
389,389,Diu,20.71,70.98,76.86,45,0,11.25,IN,1604362328,
444,444,Alice,27.75,-98.07,75.20,36,1,4.70,US,1604362354,


In [10]:
lat = hotel_df['Lat']
lng= hotel_df['Lng']
hotel_name=[]

# geocoordinates
# target_coordinates = f'{lat},{lng}'
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
#     "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    city_name = row['City']
    
    params['location'] = f'{lat},{lng}'
    
#     print(f'Retrieving Results for Index {index}:{city_name}.')
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    try: 
        print(f"The closest hotel in {city_name} is {response['results'][0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
        hotel_name.append(response['results'][0]['name'])
        
    except (KeyError, IndexError):
        print("No hotel found")
    
#     print('-------------')
    

The closest hotel in Hobart is MACq 01 Hotel.
The closest hotel in Verāval is THE SOMNATH GATEWAY.
The closest hotel in Formosa is Howard Johnson Hotel And Casino Formosa.
The closest hotel in Santa Rosalía is Hotel Francés.
The closest hotel in Alice Springs is DoubleTree by Hilton Hotel Alice Springs.
The closest hotel in Broken Hill is Royal Exchange Hotel.
The closest hotel in Laguna is Hilton Garden Inn Sacramento Elk Grove.
The closest hotel in Diu is Hotel TGH - The Grand Highness.
The closest hotel in Alice is Hampton Inn Alice.


In [11]:
# add hotel name to hotel df
hotel_df['Hotel Name']=hotel_name
hotel_df

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
20,20,Hobart,-42.88,147.33,77.00,43,0,10.29,AU,1604361923,MACq 01 Hotel
87,87,Verāval,20.90,70.37,78.69,41,0,13.76,IN,1604362184,THE SOMNATH GATEWAY
160,160,Formosa,-26.18,-58.18,77.00,44,0,5.82,AR,1604362037,Howard Johnson Hotel And Casino Formosa
187,187,Santa Rosalía,27.32,-112.28,82.60,35,0,2.62,MX,1604362233,Hotel Francés
224,224,Alice Springs,-23.70,133.88,89.60,11,0,11.41,AU,1604362050,DoubleTree by Hilton Hotel Alice Springs
335,335,Broken Hill,-31.95,141.43,82.40,20,0,14.99,AU,1604362301,Royal Exchange Hotel
342,342,Laguna,38.42,-121.42,82.99,18,1,3.11,US,1604362305,Hilton Garden Inn Sacramento Elk Grove
389,389,Diu,20.71,70.98,76.86,45,0,11.25,IN,1604362328,Hotel TGH - The Grand Highness
444,444,Alice,27.75,-98.07,75.20,36,1,4.70,US,1604362354,Hampton Inn Alice


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
#Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{HotelName}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

for index, row in hotel_df.iterrows():
    hotel_name = row['Hotel Name']
    city = row['City']
    country = row['Country']
    lat = row['Lat']
    lng = row['Lng']
    print(info_box_template.format(City=city, Country=country, HotelName=hotel_name))
    


<dl>
<dt>Name</dt><dd>MACq 01 Hotel</dd>
<dt>City</dt><dd>Hobart</dd>
<dt>Country</dt><dd>AU</dd>
</dl>


<dl>
<dt>Name</dt><dd>THE SOMNATH GATEWAY</dd>
<dt>City</dt><dd>Verāval</dd>
<dt>Country</dt><dd>IN</dd>
</dl>


<dl>
<dt>Name</dt><dd>Howard Johnson Hotel And Casino Formosa</dd>
<dt>City</dt><dd>Formosa</dd>
<dt>Country</dt><dd>AR</dd>
</dl>


<dl>
<dt>Name</dt><dd>Hotel Francés</dd>
<dt>City</dt><dd>Santa Rosalía</dd>
<dt>Country</dt><dd>MX</dd>
</dl>


<dl>
<dt>Name</dt><dd>DoubleTree by Hilton Hotel Alice Springs</dd>
<dt>City</dt><dd>Alice Springs</dd>
<dt>Country</dt><dd>AU</dd>
</dl>


<dl>
<dt>Name</dt><dd>Royal Exchange Hotel</dd>
<dt>City</dt><dd>Broken Hill</dd>
<dt>Country</dt><dd>AU</dd>
</dl>


<dl>
<dt>Name</dt><dd>Hilton Garden Inn Sacramento Elk Grove</dd>
<dt>City</dt><dd>Laguna</dd>
<dt>Country</dt><dd>US</dd>
</dl>


<dl>
<dt>Name</dt><dd>Hotel TGH - The Grand Highness</dd>
<dt>City</dt><dd>Diu</dd>
<dt>Country</dt><dd>IN</dd>
</dl>


<dl>
<dt>Name</dt><dd>Ham

In [18]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))